In [ ]:
from huggingface_hub import login

# Insira seu token de acesso pessoal
login("Your_HuggingFace_API_Key",add_to_git_credential=True)


# Manual

In [ ]:
import torch
from transformers import pipeline

model_id = "meta-llama/Llama-3.2-3B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.float32,
    device = 'cuda',
)



In [ ]:
codigo = """

"""

messages = [
    {"role": "system", "content": "Você é um bot que explica códigos em python"},
    {"role": "user", "content": f"Me diga o que o seguinte código faz: \n{codigo}"},
]
outputs = pipe(
    messages,
    max_new_tokens=256,
)

# Melhorando a legibilidade do print
response = outputs[0]["generated_text"]



In [ ]:
messages = [
    {"role": "system", "content": "Você é um bot que recebe explicacões de códigos em python e gera documentações em markdown a partir desses comentários."},
    {"role": "user", "content": f"{response[2]}"},
]
outputs = pipe(
    messages,
    max_new_tokens=1024,
)

# Melhorando a legibilidade do print
response_doc = outputs[0]["generated_text"]

In [ ]:
# Iterar sobre a lista e formatar os itens
response_list = response_doc  # Substitua por sua variável exata
formatted_response = ""

for item in response_list:
    # Adicionar cada elemento da lista como um bloco formatado
    if isinstance(item, dict):
        # Formatando dicionário como JSON legível
        import json
        formatted_response += f"```json\n{json.dumps(item, indent=2, ensure_ascii=False)}\n```\n\n"
    else:
        # Adicionar elementos simples diretamente
        formatted_response += f"{str(item)}\n\n"

# Criar um Markdown-friendly string
markdown_response = f"""
# Resposta do Modelo

{formatted_response}
"""

print(markdown_response)


In [ ]:
# Extraindo a documentação gerada pelo assistente
documentation_content = ""

for item in response_doc:
    if item["role"] == "assistant":
        documentation_content += item["content"]

# Salvando ou adicionando ao arquivo Markdown
output_file_path = "documentacao_codigo_test.md"

# Abrir o arquivo em modo de adição ("a") para não sobrescrever o conteúdo existente
with open(output_file_path, "a", encoding="utf-8") as file:
    # Adicionar separadores para distinguir diferentes seções, se necessário
    file.write("\n\n---\n\n")  # Adiciona uma linha horizontal (Markdown: "---")
    file.write(documentation_content)

print(f"Conteúdo adicionado ao arquivo: {output_file_path}")


# Automatizado

In [1]:
import torch
from transformers import pipeline

model_id = "meta-llama/Llama-3.2-3B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.float32,
    device = 'cuda',
)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


In [ ]:
import nbformat
from tqdm import tqdm
import json

# Função para filtrar células de código relevantes
def extract_code_cells(notebook_path):
    with open(notebook_path, "r", encoding="utf-8") as f:
        nb = nbformat.read(f, as_version=4)
    
    code_cells = []
    for cell in nb["cells"]:
        if cell["cell_type"] == "code":
            # Ignorar células vazias ou com apenas comentários
            code = cell["source"].strip()
            if code and not all(line.strip().startswith("#") for line in code.splitlines()):
                code_cells.append(code)
    return code_cells

# Função para processar cada célula de código com o modelo
def process_code_with_model(code_cells, pipe):
    markdown_responses = []
    for code in tqdm(code_cells, desc="Processando células de código", unit="célula"):
        # Alimentar o código para o modelo
        messages = [
            {"role": "system", "content": "Você é um bot que explica códigos em python"},
            {"role": "user", "content": f"Me diga o que o seguinte código faz: \n{code}"},
        ]
        outputs = pipe(messages, max_new_tokens=256)
        response = outputs[0]["generated_text"]

        # Gerar explicações em Markdown
        messages = [
            {"role": "system", "content": "Você é um bot que recebe explicacões de códigos em python e gera documentações em markdown a partir desses comentários."},
            {"role": "user", "content": response},
        ]
        outputs = pipe(messages, max_new_tokens=1024)
        markdown_response = outputs[0]["generated_text"]
        markdown_responses.append(markdown_response)
    return markdown_responses

# Função para salvar em um arquivo Markdown
def save_to_markdown_file(markdown_responses, output_file_path):
    with open(output_file_path, "a", encoding="utf-8") as file:
        for markdown_response in markdown_responses:
            # Extrair apenas o conteúdo do "role": "assistant"
            if isinstance(markdown_response, list):
                assistant_responses = [
                    msg.get("content", "") for msg in markdown_response if msg.get("role") == "assistant"
                ]
                markdown_response = "\n".join(assistant_responses)
            elif isinstance(markdown_response, dict):
                markdown_response = markdown_response.get("content", "") if markdown_response.get("role") == "assistant" else ""
            
            # Adiciona separadores e salva no arquivo
            file.write("\n\n---\n\n")  # Separador entre seções
            file.write(markdown_response)

# Caminho do notebook e arquivo de saída
notebook_path = "channel_segmentation_presal_clean_Full_semi_supervised_CI.ipynb"  # Substitua pelo caminho do seu notebook
output_file_path = "documentacao_codigo_final.md"

# Extração e processamento
code_cells = extract_code_cells(notebook_path)

if not code_cells:
    print("Nenhuma célula de código relevante encontrada.")
else:
    print(f"{len(code_cells)} células de código encontradas para processar.")
    markdown_responses = process_code_with_model(code_cells, pipe)
    save_to_markdown_file(markdown_responses, output_file_path)
    print(f"Documentação salva em: {output_file_path}")


45 células de código encontradas para processar.


Processando células de código:   0%|          | 0/45 [00:00<?, ?célula/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Processando células de código:   2%|▏         | 1/45 [06:00<4:24:18, 360.42s/célula]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Processando células de código:   4%|▍         | 2/45 [12:08<4:21:26, 364.79s/célula]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Processando células de código:   7%|▋         | 3/45 [22:17<5:33:24, 476.29s/célula]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Processando células de código:   9%|▉         | 4/45 [30:51<5:35:39, 491.20s/célula]Setting `pad_token_id` 

TypeError: write() argument must be str, not list